In [15]:
import pandas as pd
parent_dir = "C:\\Users\\engin\\Documents\\Github\\Energy\\"
exported_data_dir = parent_dir + "ExportedData\\"
df = pd.read_csv(exported_data_dir + "total_energies_of_decay_chains.csv", 
                on_bad_lines = 'warn')
isotopes_for_energies = [x for x in df.columns]
df = df.transpose()
df

,0
100Ag,1055.30
100Cd,1518.64
100In,2857.78
100Kr,17239.11
100Nb,2414.00
...,...
99Tc,84.60
99Y,5910.06
99Zr,3558.24
9C,0.00


In [16]:
df = df.rename(columns={0:"Expected Energy (keV per atom of parent isotope)"})
df

,Expected Energy (keV per atom of parent isotope)
100Ag,1055.30
100Cd,1518.64
100In,2857.78
100Kr,17239.11
100Nb,2414.00
...,...
99Tc,84.60
99Y,5910.06
99Zr,3558.24
9C,0.00


In [17]:
df["Isotope"] = isotopes_for_energies
df

,Expected Energy (keV per atom of parent isotope),Isotope
100Ag,1055.30,100Ag
100Cd,1518.64,100Cd
100In,2857.78,100In
100Kr,17239.11,100Kr
100Nb,2414.00,100Nb
...,...,...
99Tc,84.60,99Tc
99Y,5910.06,99Y
99Zr,3558.24,99Zr
9C,0.00,9C


In [18]:
df["A"] = [float("".join([y for y in x if y.isdigit()])) for x in df["Isotope"]]

In [19]:
df["Specific Energy (keV per Mass Number)"] = (
    df["Expected Energy (keV per atom of parent isotope)"] / df["A"]
)

In [20]:
df

,Expected Energy (keV per atom of parent isotope),Isotope,A,Specific Energy (keV per Mass Number)
100Ag,1055.30,100Ag,100.0,10.553000
100Cd,1518.64,100Cd,100.0,15.186400
100In,2857.78,100In,100.0,28.577800
100Kr,17239.11,100Kr,100.0,172.391100
100Nb,2414.00,100Nb,100.0,24.140000
...,...,...,...,...
99Tc,84.60,99Tc,99.0,0.854545
99Y,5910.06,99Y,99.0,59.697576
99Zr,3558.24,99Zr,99.0,35.941818
9C,0.00,9C,9.0,0.000000


In [27]:
import plotly
import plotly.express as px
fig = px.scatter(df, x = "Specific Energy (keV per Mass Number)", 
y = "Expected Energy (keV per atom of parent isotope)",
hover_name="Isotope", log_y=True,
title = "Energy Produced by Entire Decay Chain for Different Parent Isotopes")
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('<b>%{hovertext}</b><br><br>Spe' ... 't isotope)=%{y}<extra></extra>'),
              'hovertext': array(['100Ag', '100Cd', '100In', ..., '99Zr', '9C', '9Li'], dtype=object),
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'showlegend': False,
              'type': 'scattergl',
              'x': array([ 10.553     ,  15.1864    ,  28.5778    , ...,  35.94181818,
                            0.        , 634.44444444]),
              'xaxis': 'x',
              'y': array([1055.3 , 1518.64, 2857.78, ..., 3558.24,    0.  , 5710.  ]),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Energy Produced by Entire Decay Chain for Different Parent Isotopes'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Specific Energy (keV per Mass Number)'}},
               'yaxis': {'anchor': 'x',
                         'domain': [0.0, 1.0],
                         'title': {'text': 'Expected Energy (keV per atom of parent isotope)'},
                         'type': 'log'}}
})

In [ ]:
fig.write_html("C:\\Users\\engin\\Documents\\Github\\plots\\energy_of_decay_chains.html")

In [ ]:
df = pd.read_csv("energies.csv").iloc[:,1:]

This next cell is written in Julia

In [9]:
file_path = "C:\\Users\\engin\\Documents\\GitHub\\Energy\\ImportedData\\IsotopicAbundances.txt"
file_as_array = split(open(f->read(f, String), file_path), "\n")
file_as_array
#I want to make a dictionary where they keys are the isotope and the values are the abundance
isotope_abundances = Dict()
function add_to_dict(start_index)
    try
        atomic_number = parse(Int64, file_as_array[start_index][17:end-1])
        mass_number = parse(Int64, file_as_array[start_index+2][14:end-1])
        #if the abundance is 0, the isotope is not naturally occuring
        abundance = file_as_array[start_index+4][23:end-1]
        if occursin("(", abundance)
            abundance = split(abundance, "(")[1]
        end
        abundance = tryparse(Float64, abundance)
        if abundance == nothing
            isotope_abundances[(atomic_number, mass_number)] = 0
        else    
            isotope_abundances[(atomic_number, mass_number)] = abundance
        end
    catch
        print("Error in line ", start_index)
        return
    end
end
for start_index in 1:8:length(file_as_array)-1
    add_to_dict(start_index)
end
#works :) 
naturally_occuring_isotopes = [key for key in keys(isotope_abundances)
                                if isotope_abundances[key] > 0]
#Isotopes are currently writtenin (z, a) format. Now to make them easier to read.
element_symbols = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", 
"Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca", "Sc", "Ti", "V", 
"Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", 
"Kr", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", 
"Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", 
"Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", 
"Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", 
"Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", 
"Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", 
"Hs", "Mt", "Ds", "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"]
natural_isotopes = []
for i in 1:length(naturally_occuring_isotopes)
    z, a = naturally_occuring_isotopes[i]
    element = element_symbols[z]
    push!(natural_isotopes, element * "-" * string(a))
end
natural_isotopes


288-element Vector{Any}:
 "Zn-68"
 "Mo-97"
 "Se-80"
 "Zr-94"
 "Ru-102"
 "Cd-108"
 "Sm-147"
 "S-33"
 "Fe-54"
 "U-238"
 "Pt-195"
 "W-184"
 "Mo-95"
 ⋮
 "O-17"
 "Pd-108"
 "Cs-133"
 "Ir-191"
 "Te-124"
 "Cd-106"
 "Ir-193"
 "Os-187"
 "Rb-85"
 "Ru-100"
 "Mg-25"
 "Dy-158"

In [22]:
using DelimitedFiles
writedlm("naturally_occuring_isotopes.txt", natural_isotopes)

Now switching back to Python